## Arch_NN. Autumn 2022.
### TASK 5. Sorokin Mikhail group J42322c.

# Walker2D

You aim in this task is to train the agent to win in Walker2D game with Actor-Critic, Advantage Actor Critic (A2C), Trust-region Policy Optimization (TRPO) or Proximal Policy Optimization (PPO). 
To solve the task feel free to transform the state and reward from the environment.

**Scoring**: Calculating the average reward for 50 episodes. You goal is to gain more than 1000 points.

**Submission format**: send you notebook and trained model in **zipped** folder.


In [6]:
import pybullet_envs
from gym import make
import numpy as np
import torch
from torch import nn
from torch.distributions import Normal
from torch.nn import functional as F
from torch.optim import Adam
import random
import os

In [7]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [8]:
torch.set_default_tensor_type(torch.cuda.FloatTensor)

# Train

In [9]:
ENV_NAME = "Walker2DBulletEnv-v0"

LAMBDA = 0.95
GAMMA = 0.99

# With the default parameters I only managed to get ~700 reward from   
# 1000 iters, so ACTOR_LR, CRITIC_LR, BATCHES_PER_UPDATE are changed:
ACTOR_LR = 0.00004
CRITIC_LR = 0.00002

CLIP = 0.2
ENTROPY_COEF = 1e-2
BATCHES_PER_UPDATE = 2048
BATCH_SIZE = 64

MIN_TRANSITIONS_PER_UPDATE = 2048
MIN_EPISODES_PER_UPDATE = 4

ITERATIONS = 1000

    
def compute_lambda_returns_and_gae(trajectory):
    lambda_returns = []
    gae = []
    last_lr = 0.
    last_v = 0.
    for _, _, r, _, v in reversed(trajectory):
        ret = r + GAMMA * (last_v * (1 - LAMBDA) + last_lr * LAMBDA)
        last_lr = ret
        last_v = v
        lambda_returns.append(last_lr)
        gae.append(last_lr - v)
    
    # Each transition contains state, action, old action probability, value estimation and advantage estimation
    return [(s, a, p, v, adv) for (s, a, _, p, _), v, adv in zip(trajectory, reversed(lambda_returns), reversed(gae))]
    


class Actor(nn.Module):
    def __init__(self, state_dim, action_dim):
        super().__init__()
        ''' YOUR CODE HERE '''
        self.model = nn.Sequential(nn.Linear(state_dim, 256),
                                   nn.ELU(),
                                   nn.Linear(256, 256),
                                   nn.ELU(),
                                   nn.Linear(256, action_dim)
                                   )
        # Advice: use same log_sigma for all states to improve stability
        # You can do this by defining log_sigma as nn.Parameter(torch.zeros(...))
        self.sigma = nn.Parameter(torch.ones(action_dim, device=device))

    def compute_proba(self, state, action):
        # Returns probability of action according to current policy and distribution of actions
        ''' YOUR CODE HERE '''
        mu = self.model(state)
        sigma = torch.exp(self.sigma).unsqueeze(0).expand_as(mu)
        distr = Normal(mu, sigma)
        return torch.exp(distr.log_prob(action).sum(-1)), distr
        
    def act(self, state):
        # Returns an action (with tanh), not-transformed action (without tanh) and distribution of non-transformed actions
        # Remember: agent is not deterministic, sample actions from distribution (e.g. Gaussian)
        ''' YOUR CODE HERE '''
        mu = self.model(state)
        sigma = torch.exp(self.sigma).unsqueeze(0).expand_as(mu)
        distr = Normal(mu, sigma)
        pure_action = distr.sample()
        action = torch.tanh(pure_action)
        return action, pure_action, distr
        
        
class Critic(nn.Module):
    def __init__(self, state_dim):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(state_dim, 256),
            nn.ELU(),
            nn.Linear(256, 256),
            nn.ELU(),
            nn.Linear(256, 1)
        )
        
    def get_value(self, state):
        return self.model(state)


class PPO:
    def __init__(self, state_dim, action_dim):
        self.actor = Actor(state_dim, action_dim)
        self.critic = Critic(state_dim)
        self.actor_optim = Adam(self.actor.parameters(), ACTOR_LR)
        self.critic_optim = Adam(self.critic.parameters(), CRITIC_LR)

    def update(self, trajectories):
        transitions = [t for traj in trajectories for t in traj] # Turn a list of trajectories into list of transitions
        state, action, old_prob, target_value, advantage = zip(*transitions)
        state = np.array(state)
        action = np.array(action)
        old_prob = np.array(old_prob)
        target_value = np.array(target_value)
        advantage = np.array(advantage)
        advnatage = (advantage - advantage.mean()) / (advantage.std() + 1e-8)
        
        
        for _ in range(BATCHES_PER_UPDATE):
            idx = np.random.randint(0, len(transitions), BATCH_SIZE) # Choose random batch
            s = torch.tensor(state[idx], device=device).float()
            a = torch.tensor(action[idx], device=device).float()
            op = torch.tensor(old_prob[idx], device=device).float() # Probability of the action in state s.t. old policy
            v = torch.tensor(target_value[idx], device=device).float() # Estimated by lambda-returns 
            adv = torch.tensor(advantage[idx], device=device).float() # Estimated by generalized advantage estimation 
            
            ''' YOUR CODE HERE '''
            # TODO: Update actor here            
            # TODO: Update critic here
            prob_new, distr = self.actor.compute_proba(s, a)
            entropy = distr.entropy().mean()
            ratio = prob_new / op
            actor_loss = -torch.mean(torch.minimum(ratio * adv, torch.clip(ratio, 1 - CLIP, 1 + CLIP) * adv))
            self.actor_optim.zero_grad()
            actor_loss.backward()
            self.actor_optim.step()           
            
            val = self.critic.get_value(s).flatten()
            critic_loss = F.mse_loss(val, v)
            self.critic_optim.zero_grad()
            critic_loss.backward()
            self.critic_optim.step()
            
    def get_value(self, state):
        with torch.no_grad():
            state = torch.tensor(np.array([state]), device=device).float()
            value = self.critic.get_value(state)
        return value.cpu().item()

    def act(self, state):
        with torch.no_grad():
            state = torch.tensor(np.array([state]), device=device).float()
            action, pure_action, distr = self.actor.act(state)
            prob = torch.exp(distr.log_prob(pure_action).sum(-1))
        return action.cpu().numpy()[0], pure_action.cpu().numpy()[0], prob.cpu().item()

    def save(self):
        torch.save(self.actor, "agent.pkl")


def evaluate_policy(env, agent, episodes=5):
    returns = []
    for _ in range(episodes):
        done = False
        state = env.reset()
        total_reward = 0.
        
        while not done:
            state, reward, done, _ = env.step(agent.act(state)[0])
            total_reward += reward
        returns.append(total_reward)
    return returns
   

def sample_episode(env, agent):
    s = env.reset()
    d = False
    trajectory = []
    while not d:
        a, pa, p = agent.act(s)
        v = agent.get_value(s)
        ns, r, d, _ = env.step(a)
        trajectory.append((s, pa, r, p, v))
        s = ns
    return compute_lambda_returns_and_gae(trajectory)

In [10]:
env = make(ENV_NAME)
ppo = PPO(state_dim=env.observation_space.shape[0], action_dim=env.action_space.shape[0])
state = env.reset()
episodes_sampled = 0
steps_sampled = 0
latest_reward = 0

for i in range(ITERATIONS):
    # Add a stop condition when a reward gets to 1000:
    if latest_reward < 1000:
        trajectories = []
        steps_ctn = 0

        while len(trajectories) < MIN_EPISODES_PER_UPDATE or steps_ctn < MIN_TRANSITIONS_PER_UPDATE:
            traj = sample_episode(env, ppo)
            steps_ctn += len(traj)
            trajectories.append(traj)
        episodes_sampled += len(trajectories)
        steps_sampled += steps_ctn

        ppo.update(trajectories)        

        if (i + 1) % (ITERATIONS//100) == 0:
            rewards = evaluate_policy(env, ppo, 5)
            print(f"Step: {i+1}, Reward mean: {np.mean(rewards)}, Reward std: {np.std(rewards)}, Episodes: {episodes_sampled}, Steps: {steps_sampled}")
            latest_reward = np.mean(rewards)
            ppo.save()

Step: 10, Reward mean: 26.261720153973727, Reward std: 12.911079193224099, Episodes: 1028, Steps: 20578
Step: 20, Reward mean: 54.35861523681873, Reward std: 10.757852836691757, Episodes: 1327, Steps: 41569
Step: 30, Reward mean: 59.662477453337054, Reward std: 4.253775994822421, Episodes: 1609, Steps: 62501
Step: 40, Reward mean: 83.68429264024509, Reward std: 17.78381986583689, Episodes: 1881, Steps: 83342
Step: 50, Reward mean: 99.0117590175342, Reward std: 14.094299651175609, Episodes: 2105, Steps: 104127
Step: 60, Reward mean: 112.44725655775923, Reward std: 54.51788594763969, Episodes: 2282, Steps: 125281
Step: 70, Reward mean: 146.08551028820747, Reward std: 26.928205123257076, Episodes: 2421, Steps: 147886
Step: 80, Reward mean: 198.32204781502745, Reward std: 92.52114413051369, Episodes: 2551, Steps: 169621
Step: 90, Reward mean: 145.36460915303235, Reward std: 31.140030535517923, Episodes: 2665, Steps: 191251
Step: 100, Reward mean: 174.41408071441165, Reward std: 122.4963465

# Agent

In [11]:
class Agent:
    def __init__(self):
        self.model = torch.load(__file__[:-8] + "/agent.pkl")
        
    def act(self, state):
        with torch.no_grad():
            state = torch.tensor(np.array(state)).float()
            ''' YOUR CODE HERE '''
            actions = self.model(state)
            return np.argmax(actions.cpu().numpy())

    def reset(self):
        pass